## Оглавление:
* [1 Описание данных](#One)
* [2 Загрузка данных](#Two)
* [2.1 Разделение данных на выборки](#TwoPointOne)
* [3 Исследование моделей](#Three)
* [4 Проверка модели на тестовой выборке](#Four)
* [5 Проверка на адекватность](#Five)

# Рекомендация тарифов

Анализ использования тарифных планов мобильного оператора. Нужно построить модель классификации, которая будет рекомендовать пользователям тарф, в зависимости от того как они пользуются услугами мобильного оператора.
Постройте модель с максимально большим значением *accuracy* > 0.75.

## 1 Описание данных <a class="anchor" id="One"></a>
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

    сalls — количество звонков,
    minutes — суммарная длительность звонков,
    messages — количество sms,
    mb_used — израсходованный трафик,
    is_ultra — Тариф («Ультра» — 1, «Смарт» — 0).

## 2 Загрузка данных <a class="anchor" id="Two"></a>

In [36]:
import pandas as pd
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [37]:
df = pd.read_csv('users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [39]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

### 2.1 Разделение данных на выборки <a class="anchor" id="TwoPointOne"></a>

#### Датасет разобьем два раза, сначала отделим тестовую выборку (20%), потом разделим оставшийся датасет на валидационную (20% от общего датасета) и тренировочную (60% от общего датасета.

In [40]:
df_train_valid, df_test = train_test_split(
    df, 
    test_size=0.2, 
    random_state=12345)

In [41]:
df_train, df_valid = train_test_split(
    df_train_valid, 
    test_size=0.25, 
    random_state=12345)

#### Выберем признаки и целевой признак для каждой выборки


In [42]:
features_train=df_train.drop(['is_ultra'], axis=1)
target_train=df_train['is_ultra']

features_valid=df_valid.drop(['is_ultra'], axis=1)
target_valid=df_valid['is_ultra']

features_test=df_test.drop(['is_ultra'], axis=1)
target_test=df_test['is_ultra']


#### Также выберем признаки и целевой признак для суммарной валидационной и тренировочной выборки

In [43]:
features=df_train_valid.drop(['is_ultra'], axis=1)
target=df_train_valid['is_ultra']

## 3 Исследование моделей <a class="anchor" id="Three"></a>

#### Исследуем модель логистической регрессии

In [44]:
model_logistic = LogisticRegression(random_state=12345)
model_logistic.fit(features_train, target_train)

/home/kharan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=12345, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [45]:
predict = model_logistic.predict(features_valid)

In [46]:
def model_view(model):
    """
    Функция определния accuracy_score для тренировочной, валидационной и тестовойвыборок.
    model - обученая модель
    
    """
    print('Accuracy на тренировочной выборке - {:.4f}'.format(model.score(features_train, target_train)))
    print('Accuracy на валидационной выборке - {:.4f}'.format(model.score(features_valid, target_valid)))
    print('Accuracy на тестовой выборке - {:.4f}'.format(model.score(features_test, target_test)))

#### Посчитаем accuracy score для модели логистической регрессии

In [47]:
model_view(model_logistic)

Accuracy на тренировочной выборке - 0.7510
Accuracy на валидационной выборке - 0.7263
Accuracy на тестовой выборке - 0.7589


#### Исследуем модель случайного леса
Для определения наилучших параметров модели проведем подбор max_depth и n_estimators

In [48]:
#d - max_depth, i - n_estimators
s = 0
for d in range (1,20):
    for i in range (1, 100, 5):
        model_forest = RandomForestClassifier(
            random_state=12345, 
            n_estimators=i, 
            max_depth = d)
        
        model_forest.fit(features_train, target_train)
        score = model_forest.score(features_valid, target_valid)
        #s - наилучший Accuracy, estimate - лучшее значение n_estimators, d_1 - лучшее значение max_depth
        if score > s:
            s = score
            estimate = i
            d_1 = d
            
# По результатам перебора выводится наилучшие параметры модели
print('Наилучшие результаты показали следующие гиперпараметры:')
print('n_estimate = {}, depth = {}, Accuracy = {:.4f}'.format(estimate, d_1, s))

Наилучшие результаты показали следующие гиперпараметры:
n_estimate = 76, depth = 12, Accuracy = 0.8009


Используем полученные параметры для обучения модели

In [49]:
model_forest = RandomForestClassifier(
    random_state=12345, 
    n_estimators=estimate, 
    max_depth=d_1)

model_forest.fit(features_train, target_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=76,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

#### Посчитаем accuracy score для модели слчайного леса

In [50]:
model_view(model_forest)

Accuracy на тренировочной выборке - 0.9134
Accuracy на валидационной выборке - 0.8009
Accuracy на тестовой выборке - 0.7978


#### Исследуем модель дерева решений
Подберем лучшие параметры max depth

In [51]:
s = 0
for depth in range(1, 51):
    model_tree = DecisionTreeClassifier(
        random_state=123445, 
        max_depth=depth)
    
    model_tree.fit(features_train, target_train)
    score = model_tree.score(features_valid, target_valid)
    
    if score > s:
        s = score
        m_depth = depth
# По результатам перебора выводится наилучшие параметры модели
print('max_depth = {}, Accuracy = {:.4f}'.format(depth, s))

max_depth = 50, Accuracy = 0.7729


Используем полученные параметры для обучения модели

In [52]:
model_tree = DecisionTreeClassifier(
    random_state=123445, 
    max_depth=m_depth)

model_tree.fit(features_train, target_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=123445, splitter='best')

#### Посчитаем accuracy score для модели дерева решений

In [53]:
model_view(model_tree)

Accuracy на тренировочной выборке - 0.8506
Accuracy на валидационной выборке - 0.7729
Accuracy на тестовой выборке - 0.7854


## Вывод 
Наилуший результат показала модель случайного леса с параметрами n_estimate = 76 и max_depth= 12, ее accuracy score получился равным 0,8.

## 4 Проверка модели на тестовой выборке <a class="anchor" id="Four"></a>

#### Обучим модель случайного леса с выбранными параметрами на суммарной тренировочной и валидационной выборке.

In [54]:
model_sum = RandomForestClassifier(
    random_state=12345, 
    n_estimators=estimate, 
    max_depth=d_1)

model_sum.fit(features, target)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=12, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=76,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [55]:
print('Accuracy score итоговой модели на тестовой выборке - {:.4f}'.format(
    model_sum.score(features_test, target_test)))

Accuracy score итоговой модели на тестовой выборке - 0.7947


#### Получим предсказание для тестовой выборки

In [56]:
predict = model_sum.predict(features_test)

## Вывод. 
Проверка модели показала значение accuracy score = 7,9 что больше заданного 7,5.

## 5  Проверка на адекватность <a class="anchor" id="Five"></a>

#### Для проверки используем два метода:
Проверка с помощью DummyClassifier
Проверка по F-критерия

### Проверка с помощью DummyClassifier

In [57]:
from sklearn.dummy import DummyClassifier

In [58]:
dummy_model = DummyClassifier()
dummy_model.fit(features, target)

/home/kharan/anaconda3/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


DummyClassifier(constant=None, random_state=None, strategy='warn')

In [59]:
print('Accuracy score Dummy модели на тестовой выборке - {:.4f}'.format(
    dummy_model.score(features_test, target_test)))

Accuracy score Dummy модели на тестовой выборке - 0.5459


#### Проверка показала что предсказания модели случайного леса значительно лучше чем Dummy модель.

### Проверка F-критерием

In [60]:
from sklearn.metrics import f1_score

In [61]:
f1_score(target_test, predict)

0.6094674556213017

Метрика F1_score принимает значение от 0 до 1, причем если значение близко к нулю - предсказательная способность модели сомнительная. Полученное значение говорит об адекватности выбранной модели

## Вывод
1. Полученная модель случайного леса лучше предсказывает чем Dummy модель.
2. F1_score равное 0,6 говорит об адекватности выбранной модели.

#### Считаем полученную модель адекватной для предсказания.